In [1]:
import collections

from Bio import pairwise2
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

from tf.app import use
A = use('etcbc/bhsa', hoist=globals())
Fmt, Tmt, Lmt = F, T, L

B = use('dt-ucph/sp', version='3.4.1', hoist=globals())
Fsp, Tsp, Lsp = F, T, L

C = use('etcbc/dss', version='1.9', hoist=globals())
Fdss, Tdss, Ldss = F, T, L

del F, T, L

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114891,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
PENTATEUCH_BOOKS = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy']
ALL_BOOK_NAMES = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]

In [3]:
QSP_SCROLLS = {'1Qisaa', '1QisaaI', '1QisaaII', '2Q3', '4Q13', '4Q20', '2Q7', '4Q27', '1Q4', '2Q12', '4Q37', '4Q38', '4Q38a', '4Q40', '4Q53',
               '4Q57', '2Q13', '4Q78', '4Q80', '4Q82', '4Q128', '4Q129', '4Q134', '4Q135', '4Q136',
                '4Q137', '4Q138', '4Q139', '4Q140', '4Q141', '4Q142', '4Q143', '4Q144', '4Q158', '4Q364',
                '4Q365', '4Q96', '4Q111', '4Q109', '11Q5', '11Q6', '11Q7', '11Q8'}

In [4]:
class Book:
    def __init__(self, manuscript, book_name, F, T, L):
        self.manuscript = manuscript
        self.book_name = book_name
        self.F = F
        self.T = T
        self.L = L
        if self.manuscript in {'MT', 'SP'}:
            self.verse_g_cons, self.word2char = self.prepare_book_data()
        else:
            self.verse_g_cons, self.word2char = self.prepare_dss_book_data()
        self.verse_text_dict = self.make_verse_text()
        
    def prepare_book_data(self):
        verse_g_cons = collections.defaultdict(list)
        word2char = collections.defaultdict(list)
    
        for book_node in eval(self.F + ".otype.s('book')"):
            book_name = eval(self.T + '.sectionFromNode(book_node)[0]')
            if book_name != self.book_name:
                continue
            words = eval(self.L + ".d(book_node, 'word')")
            for w in words:
                bo, ch, ve = eval(self.T + '.sectionFromNode(w)')
                g_cons = eval(self.F + '.g_cons.v(w)')
                trailer = eval(self.F + '.trailer.v(w)')
                if trailer:
                    trailer = ' '
                verse_g_cons[(bo, ch, ve)].append(g_cons + trailer)
                for char in g_cons:
                    word2char[(bo, ch, ve)].append(w)
        return verse_g_cons, word2char
    
    def prepare_dss_book_data(self):
        verse_g_cons = collections.defaultdict(list)
        word2char = collections.defaultdict(list)
        
        for scr in eval(self.F + ".otype.s('scroll')"):
            if eval(self.T + '.scrollName(scr)') == self.manuscript:
                words = eval(self.L + ".d(scr, 'word')")
                for w in words:
                    bo = eval(self.F + '.book_etcbc.v(w)')
                    if bo != self.book_name:
                        continue            
                    ch = eval(self.F + '.chapter.v(w)')
                    ve = eval(self.F + '.verse.v(w)')
                
                    g_cons = eval(self.F + '.g_cons.v(w)')
                    after = eval(self.F + '.after.v(w)')

                    if after is None:
                        after = ''
                    if g_cons:
                        verse_g_cons[(bo, int(ch), int(ve))].append(g_cons + after)
                        for char in g_cons:
                            word2char[(bo, int(ch), int(ve))].append(w)
        return verse_g_cons, word2char
    
    def make_verse_text(self):
        return {section : ''.join(g_conss).strip() for (section, g_conss) in self.verse_g_cons.items()}

In [5]:
class AllBooks:
    def __init__(self):
        self.data = {}
        

In [6]:
def align_verses(str_1, str_2):
        
    seq1 = Seq(str_1)
    seq2 = Seq(str_2)
    
    alignments = pairwise2.align.globalxx(seq1, seq2)
    
    seq1_al = (alignments[0][0]).strip(' ')
    seq2_al = (alignments[0][1]).strip(' ')
        
    return seq1_al, seq2_al

In [7]:
def make_alignments(verse_text1, verse_text2):
    alignments_dict = {}

    for section, text1 in verse_text1.items():
        try:
            text2 = verse_text2[section]
            alignment1, alignment2 = align_verses(text1, text2)
            alignments_dict[section] = (alignment1, alignment2)
        except:
            continue
    return alignments_dict

In [8]:
def collect_matching_words(alignments_dict, word2char1, word2char2):
    man1_man2_dict = collections.defaultdict(list)

    for section, (al1, al2) in alignments_dict.items():
        man1_idx = 0
        man2_idx = 0
    
        word_chars1 = word2char1[section]
        word_chars2 = word2char2[section]
        for char1, char2 in zip(al1, al2):
            if char1 not in {' ', '-'}:
                man1_word = word_chars1[man1_idx]
                man1_idx += 1
            
            if char2 not in {' ', '-'}:
                man2_word = word_chars2[man2_idx]
                man2_idx += 1
            
            if char1 not in {' ', '-'} and char2 not in {' ', '-'}:
                man1_man2_dict[man1_word].append(man2_word)
                
    return man1_man2_dict

In [9]:
def most_frequent(List):
    return max(set(List), key = List.count)

# Prepare MT and SP texts

Produce dictionary mt_sp_matches which has mt words nodes as keys and matching word numbers from SP as values.

In [10]:
# prepare mt and sp books
MANUSCRIPTS = ['MT', 'SP']
all_books = AllBooks()
for book_name in ALL_BOOK_NAMES:
    book = Book('MT', book_name, 'Fmt', 'Tmt', 'Lmt')
    all_books.data[('MT', book_name)] = book
    
    if book_name in PENTATEUCH_BOOKS:
        book = Book('SP', book_name, 'Fsp', 'Tsp', 'Lsp')
        all_books.data[('SP', book_name)] = book

# Match words

In [46]:
dat = pd.read_csv('../data/niph_hiph_pe_yod.csv', sep='\t')
dat_dss = dat[~dat.scroll.isin(['MT'])] #, 'SP'])]
scroll_book_combinations = list(set(zip(dat_dss.scroll, dat_dss.book)))
print(dat.shape, dat_dss.shape)

(2332, 36) (430, 36)


In [47]:
for manuscript, book_name in scroll_book_combinations:
    if manuscript in {'SP', 'MT'}:
        continue
    book = Book(manuscript, book_name, 'Fdss', 'Tdss', 'Ldss')
    all_books.data[(manuscript, book_name)] = book

In [48]:
def make_matching_word_dict(book_name, all_books):

    all_match_dicts = {}

    matching_book = book_name
    matching_manuscripts = [scr for scr, bo in all_books.data.keys() if bo == matching_book]

    for man1 in matching_manuscripts:
        for man2 in matching_manuscripts:
            if man1 != man2:
            
                man1_verse_texts = all_books.data[(man1, matching_book)].verse_text_dict
                man2_verse_texts = all_books.data[(man2, matching_book)].verse_text_dict

                alignments_dict = make_alignments(man1_verse_texts, man2_verse_texts)
    
                man1_word2char = all_books.data[(man1, matching_book)].word2char
                man2_word2char = all_books.data[(man2, matching_book)].word2char
 
                matching_words_dict = collect_matching_words(alignments_dict, man1_word2char, man2_word2char)

                matches = {}
                for man1_word, man2_list in matching_words_dict.items():
                    man2_word = most_frequent(man2_list)
                    matches[man1_word] = man2_word
                all_match_dicts[((man1, matching_book), (man2, matching_book))] = matches
                
    return all_match_dicts, matching_manuscripts

In [49]:
def read_dataset(file):

    return pd.read_csv(file, sep='\t')

In [50]:
class MaterData:
    def __init__(self, man1, man2, section, lex, mater_val1, mater_val2, tf_id1, tf_id2, g_cons1, g_cons2):
        self.man1 = man1
        self.man2 = man2
        self.section = section
        self.lex = lex
        self.mater_val1 = mater_val1
        self.mater_val2 = mater_val2
        self.tf_id1 = tf_id1
        self.tf_id2 = tf_id2
        self.g_cons1 = g_cons1
        self.g_cons2 = g_cons2

In [51]:
def collect_matching_cases(matching_manuscripts, matching_book, dat):

    all_mater_datasets = {}

    for man in matching_manuscripts:
        mater_data = dat[(dat.book == matching_book) & (dat.scroll == man)]
        all_mater_datasets[(man, matching_book)] = mater_data
        
    return all_mater_datasets

In [52]:
def collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, matching_book):

    manuscript_mater_match = collections.defaultdict(list)
    manuscripts = set()

    for idx, man in enumerate(matching_manuscripts):
        for idx2, man2 in enumerate(matching_manuscripts):
            if idx < idx2:
            
                matching_ids = all_match_dicts[((man, matching_book), (man2, matching_book))]
                man_data = all_mater_datasets[(man, matching_book)]
                man2_data = all_mater_datasets[(man2, matching_book)]
                for _, row in man_data.iterrows():
                
                    tf_id = row.tf_id
                    lex, typ, has_vl = row.lex, row.type, row.has_vowel_letter
                    g_cons1 = row.g_cons
                    section = (row.book, row.chapter, row.verse)
                    if man == 'SP':
                        tf_id = tf_id - 100000

                    matching_tf_id = matching_ids.get(tf_id, None)
                
                    if not matching_tf_id:
                        continue
                    
                    if man2 == 'SP':
                        matching_tf_id = matching_tf_id + 100000
                
                    man2_row = man2_data[(man2_data.tf_id == matching_tf_id) & (man2_data.lex == lex) & (man2_data.type == typ)]
                    
                    if not man2_row.shape[0]:
                        continue
                    has_vl2 = man2_row.has_vowel_letter.iloc[0]
                    g_cons2 = man2_row.g_cons.iloc[0]
                 
                    mater_data = MaterData(man, man2, section, lex, has_vl, has_vl2, tf_id, matching_tf_id, g_cons1, g_cons2)
                
                    manuscript_mater_match[man].append(mater_data)
                    manuscripts.add(man)
                    manuscripts.add(man2)
                    
    return manuscript_mater_match, manuscripts

In [53]:
def register_similarities_with_mt(manuscripts, mt_ids, manuscript_mater_match):

    mater_value_dict = {0: -1,
                    1: 1}

    mater_match_array = np.zeros((len(manuscripts), len(mt_ids)))

    for dat_object in manuscript_mater_match['MT']:
        print(dat_object.g_cons1, dat_object.g_cons2)
        
        other_man = dat_object.man2
        mt_tfid = dat_object.tf_id1
        other_man_tfid = dat_object.tf_id2
    
        mt_mater = dat_object.mater_val1
        other_man_mater = dat_object.mater_val2
    
        mt_idx = man2idx['MT']
        other_man_idx = man2idx[other_man]
    
        mt_mater_value = mater_value_dict[mt_mater]
        other_man_mater_value = mater_value_dict[other_man_mater]
    
        mt_tf_id = mt_tf2idx[mt_tfid]
    
        mater_match_array[mt_idx, mt_tf_id] = mt_mater_value
        mater_match_array[other_man_idx, mt_tf_id] = other_man_mater_value
        
    return mater_match_array

In [54]:
def get_parallels(manuscript_mater_match, hif_match_dict):
    for scroll in manuscript_mater_match.keys():
        for dat_object in manuscript_mater_match[scroll]:
            man1 = dat_object.man1
            man2 = dat_object.man2
            if man1 == 'MT' or man2 == 'MT':
            
                hif_match_dict[dat_object.tf_id2] = dat_object.tf_id1
                hif_match_dict[dat_object.tf_id1] = dat_object.tf_id2
        
    return hif_match_dict


In [55]:
def count_parallel_cases(mater_match_array):

    mater_arr = np.zeros((2, mater_match_array.shape[1]))

    for col_idx in range(mater_match_array.shape[1]):
        col = mater_match_array[:, col_idx]
        col_counts = collections.Counter(col)
        with_vowel_count = col_counts.get(1, 0)
        without_vowel_count = col_counts.get(-1, 0)
        mater_arr[0, col_idx] = with_vowel_count
        mater_arr[1, col_idx] = without_vowel_count
        
    return mater_arr

In [56]:
file_name = '../data/niph_hiph_pe_yod.csv'
all_bib_books = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]
dat = read_dataset(file_name)

hif_match_dict = {}

for book in all_bib_books:
    #if book != 'Isaiah':
    #    continue
    print(book)
    all_match_dicts, matching_manuscripts = make_matching_word_dict(book, all_books)
    all_mater_datasets = collect_matching_cases(matching_manuscripts, book, dat)
    manuscript_mater_match, manuscripts = collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, book)
    
    hif_match_dict = get_parallels(manuscript_mater_match, hif_match_dict)
    print(len(hif_match_dict))
    

Genesis
14
Exodus
88
Leviticus
102
Numbers
106
Deuteronomy
157
Joshua
163
Judges
163
1_Samuel
171
2_Samuel
185
1_Kings
187
2_Kings
187
Isaiah
507
Jeremiah
519
Ezekiel
525
Hosea
531
Joel
540
Amos
546
Obadiah
550
Jonah
558
Micah
568
Nahum
570
Habakkuk
574
Zephaniah
586
Haggai
586
Zechariah
588
Malachi
594
Psalms
708
Job
708
Proverbs
710
Ruth
714
Song_of_songs
714
Ecclesiastes
714
Lamentations
718
Esther
718
Daniel
718
Ezra
718
Nehemiah
718
1_Chronicles
718
2_Chronicles
718


In [57]:
hif_match_dict

{1937945: 205,
 205: 1937945,
 1938197: 449,
 449: 1938197,
 1938532: 1769,
 1769: 1938532,
 1941098: 23061,
 23061: 1941098,
 1939392: 22292,
 22292: 1939392,
 1939859: 24304,
 24304: 1939859,
 1940186: 27411,
 27411: 1940186,
 2064055: 35812,
 35812: 2091017,
 2065264: 35949,
 35949: 2064929,
 2065361: 36039,
 36039: 2089933,
 2065426: 36099,
 36099: 2089995,
 2103113: 35949,
 2070741: 35949,
 1936043: 29854,
 29854: 1936043,
 1936065: 29875,
 29875: 1936065,
 1936086: 29895,
 29895: 1936086,
 1936881: 31382,
 31382: 2090505,
 2088548: 35666,
 35666: 1949458,
 2088637: 35753,
 35753: 2077275,
 2088697: 35812,
 2088837: 35949,
 2069427: 35949,
 1952368: 31712,
 31712: 1952368,
 1952386: 31729,
 31729: 1952386,
 1954274: 34361,
 34361: 1954274,
 1955964: 38499,
 38499: 1955964,
 1960524: 46737,
 46737: 1960524,
 1961121: 48012,
 48012: 1961121,
 1943086: 34867,
 34867: 1943086,
 1943505: 37421,
 37421: 1943505,
 2102358: 35812,
 2102499: 35949,
 2102593: 36039,
 2102651: 36099,
 209050

In [58]:
len(hif_match_dict)

718

In [59]:
all_scrolls = set(dat_dss.scroll) 
non_qsp_scrolls = all_scrolls.difference(QSP_SCROLLS)
non_qsp_data = dat_dss[dat_dss.scroll.isin(non_qsp_scrolls)]
qsp_data = dat_dss[~dat_dss.scroll.isin(non_qsp_scrolls)]
non_qsp_pent = non_qsp_data[non_qsp_data.book.isin(PENTATEUCH_BOOKS)]
non_qsp_pent.shape

(84, 36)

In [63]:
n = 0
full_count = collections.defaultdict(int)
scrolls = set()
scroll_counts = collections.defaultdict(int)

for index, row in non_qsp_data.iterrows():
    tf_id = row.tf_id, 
    scroll = row.scroll, 
    has_vl = row.has_vowel_letter
    book = row.book
    g_cons = row.g_cons
    lex = row.lex
    prefix = row.prefix
    
    if tf_id[0] in hif_match_dict:
        corr_id = hif_match_dict[tf_id[0]]
        corr_row = dat[dat.tf_id == corr_id]
        
        if has_vl == 0 and corr_row.has_vowel_letter.iloc[0] == 1:
            print(scroll, corr_row.scroll.iloc[0], row.chapter, row.verse, row.lex, corr_row.vs.iloc[0], corr_row.vt.iloc[0], g_cons, corr_row.g_cons.iloc[0], 
              book, prefix, corr_row.prefix.iloc[0], has_vl, corr_row.has_vowel_letter.iloc[0])
        n += 1
        
        mt_full = corr_row.has_vowel_letter.iloc[0]
        scroll_full = has_vl
        full_count[(scroll_full, mt_full)] += 1
        scrolls.add(scroll)
        scroll_counts[scroll] += 1
        
    else:
        print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
        
    
print(n)
print(full_count)
print(len((scrolls)))
print(scroll_counts)

('1Q8',) MT 38 20 JC<[ hif infc HCJ<NJ HWCJ<NJ Isaiah L L 0 1
('1Q8',) MT 63 6 JRD[ hif impf >RJDH >WRJD Isaiah W W 0 1
('4Q31',) MT 3 26 JSP[ hif impf TSP TWSP Deuteronomy nan nan 0 1
('4Q33',) MT 21 4 JRD[ hif perf HRJDW HWRDW Deuteronomy W W 0 1
('4Q35',) Deuteronomy 33 19 JDH[ HDW
('4Q51',) MT 10 8 JD<[ hif perf HD<TJ HWD<TJ 1_Samuel W W 0 1
('4Q76',) MT 2 17 JG<[ hif perf HG<TM HWG<TM Malachi W nan 0 1
('4Q76',) MT 2 17 JG<[ hif perf HGW<NW HWG<NW Malachi nan nan 0 1
('4Q83',) MT 67 6 JDH[ hif impf JDWKH JWDWK Psalms W nan 0 1
('4Q86',) MT 104 14 JY>[ hif infc HYJ> HWYJ> Psalms L L 0 1
('4Q87',) MT 76 12 JBL[ hif impf JBJLW JWBJLW Psalms nan nan 0 1
('4Q88',) MT 107 13 JC<[ hif impf JCJ<M JWCJ<M Psalms W nan 0 1
('4Q88',) Psalms 107 24 JDH[ JWDW
('4Q88',) Psalms 107 28 JC<[ JCJ<M
('4Q88',) MT 107 36 JCB[ hif wayq JCB JWCB Psalms W W 0 1
('4Q130',) MT 13 9 JY>[ hif perf HY>NW HWY>K Exodus nan nan 0 1
('4Q130',) MT 13 14 JY>[ hif perf HY>NW HWYJ>NW Exodus nan nan 0 1
('4Q130',) MT 1

In [61]:
sect = ('Psalms', 107, 28)

node = Tmt.nodeFromSection(sect)
for w in Lmt.d(node, 'word'):
    print(w, Fmt.lex.v(w), Fmt.g_cons.v(w), Fmt.vs.v(w))

329708 W W NA
329709 Y<Q[ JY<QW qal
329710 >L >L NA
329711 JHWH/ JHWH NA
329712 B B NA
329713 H  NA
329714 YR/ YR NA
329715 L LHM NA
329716 W W NA
329717 MN M NA
329718 MYWQH/ MYWQTJHM NA
329719 JY>[ JWYJ>M hif
